In [1]:
import torch
import torch.nn as nn
import torch.optim as optim
print(torch.__version__)
print(torch.cuda.is_available())

2.1.2+cu121
True


In [2]:
from vit_pytorch import ViT

In [3]:
from __future__ import print_function
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import torch
import torch.nn as nn
import torch.optim as optim
from linformer import Linformer
from PIL import Image
from torch.optim.lr_scheduler import StepLR
from tqdm.notebook import tqdm
from vit_pytorch.efficient import ViT
from sklearn.metrics import roc_curve, roc_auc_score
from sklearn.metrics import confusion_matrix
import torch.utils.data as data
import torchvision
from torchvision.transforms import ToTensor
torch.cuda.is_available()

True

In [4]:
import torchvision.transforms as transforms

In [5]:
# Hyperparameters:
batch_size = 8 
epochs = 20
lr = 3e-5
gamma = 0.7
seed = 142
IMG_SIZE = 224
patch_size = 16
num_classes = 2

In [6]:
transform = transforms.Compose([
    transforms.Resize((IMG_SIZE, IMG_SIZE)),  # Resize all images to the same size
    transforms.ToTensor()
])

In [7]:
# Load datasets with the transform
train_ds = torchvision.datasets.ImageFolder(r'C:\Mine\Work\Uzi\Signature_Verification\train', transform=transform)
valid_ds = torchvision.datasets.ImageFolder(r'C:\Mine\Work\Uzi\Signature_Verification\validation', transform=transform)
test_ds = torchvision.datasets.ImageFolder(r'C:\Mine\Work\Uzi\Signature_Verification\test', transform=transform)

In [8]:
# Data Loaders:
train_loader = data.DataLoader(train_ds, batch_size=batch_size, shuffle=True,  num_workers=1)
valid_loader = data.DataLoader(valid_ds, batch_size=batch_size, shuffle=True,  num_workers=1)
test_loader  = data.DataLoader(test_ds, batch_size=batch_size, shuffle=True, num_workers=1)

In [9]:
torch.cuda.empty_cache()

In [10]:
# Training device:
device = 'cuda'

# Linear Transformer:
efficient_transformer = Linformer(dim=128, seq_len=64+1, depth=12, heads=8, k=64)

# Vision Transformer Model: 
model = ViT(dim=128, image_size=128, patch_size=patch_size, num_classes=num_classes, transformer=efficient_transformer, channels=3).to(device)

# loss function
criterion = nn.CrossEntropyLoss()

# Optimizer
optimizer = optim.Adam(model.parameters(), lr=lr)

# Learning Rate Scheduler for Optimizer:
scheduler = StepLR(optimizer, step_size=1, gamma=gamma)

In [11]:
efficient_transformer = Linformer(
    dim=128,
    seq_len=(IMG_SIZE // patch_size) ** 2 + 1,  # Adjusting seq_len to match the number of patches
    depth=12,
    heads=8,
    k=64
)

model = ViT(
    dim=128,
    image_size=IMG_SIZE,
    patch_size=patch_size,
    num_classes=num_classes,
    transformer=efficient_transformer,
    channels=3
).to(device)

In [12]:
scaler = torch.cuda.amp.GradScaler()

In [13]:
from torch.cuda.amp import GradScaler
scaler = GradScaler()

In [16]:
for epoch in range(epochs):
    epoch_loss = 0
    epoch_accuracy = 0

    for data, label in tqdm(train_loader):
        data, label = data.to(device), label.to(device)

        # Zero the gradients
        optimizer.zero_grad()

        # Forward pass with autocast
        with torch.cuda.amp.autocast():
            output = model(data)
            loss = criterion(output, label)

        # Backward pass on the scaled loss
        scaler.scale(loss).backward()

        # Step the optimizer and update the scaler
        scaler.step(optimizer)
        scaler.update()

        # Zero the gradients after optimizer step
        optimizer.zero_grad()

        acc = (output.argmax(dim=1) == label).float().mean()
        epoch_accuracy += acc / len(train_loader)
        epoch_loss += loss.item() / len(train_loader)
    epoch_val_accuracy = 0
    epoch_val_loss = 0
    with torch.no_grad():
        for data, label in valid_loader:
            data, label = data.to(device), label.to(device)

            # Use autocast for the forward pass in validation
            with torch.cuda.amp.autocast():
                val_output = model(data)
                val_loss = criterion(val_output, label)

            acc = (val_output.argmax(dim=1) == label).float().mean()
            epoch_val_accuracy += acc / len(valid_loader)
            epoch_val_loss += val_loss.item() / len(valid_loader)

    print(
        f"Epoch : {epoch+1} - loss : {epoch_loss:.4f} - acc: {epoch_accuracy:.4f} - val_loss : {epoch_val_loss:.4f} - val_acc: {epoch_val_accuracy:.4f}\n"
    )

  0%|          | 0/897 [00:00<?, ?it/s]

AssertionError: No inf checks were recorded for this optimizer.